In [71]:
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
import re

In [1]:
s_loc = './data/sample/'
d_sets = ['Indian-Male-Names.csv','Indian-Female-Names.csv']
d_sets

['Indian-Male-Names.csv', 'Indian-Female-Names.csv']

In [2]:
map_info = {'gender': {0:'Male',1:'Female'}, 'race' : {0:'black',1:'hispanic',2:'white',3:'indian'}}
map_info

{'gender': {0: 'Male', 1: 'Female'},
 'race': {0: 'black', 1: 'hispanic', 2: 'white', 3: 'indian'}}

In [5]:
ind_m = pd.read_csv(s_loc+d_sets[0])
ind_f = pd.read_csv(s_loc+d_sets[1])
ind_f.head()

,name,gender,race,length
0,shivani,1,3,7
1,isha,1,3,4
2,smt shyani devi,1,3,15
3,divya,1,3,5
4,mansi,1,3,5


In [6]:
ind_m.head()

,name,gender,race,length
0,barjraj,0,3,7
1,ramdin verma,0,3,12
2,sharat,0,3,6
3,birender,0,3,8
4,amit,0,3,4


In [7]:
ind = ind_m[:6800].append(ind_f)
ind.head()

,name,gender,race,length
0,barjraj,0,3,7
1,ramdin verma,0,3,12
2,sharat,0,3,6
3,birender,0,3,8
4,amit,0,3,4


In [8]:
ind.gender.value_counts()

0    6800
1    5685
Name: gender, dtype: int64

In [9]:
len(ind)

12485

## Actual value

In [10]:
y = ind.gender
len(y)

12485

In [34]:
X = ind.name.values
type(X)

numpy.ndarray

## Test-Train split

In [31]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [36]:
X_train.shape

(9988,)

In [37]:
y_test.shape

(2497,)

## Features

In [63]:
def preprocess(word):
    return re.sub(r'[^a-z.\s]+','',str(word))

def name_to_list(names):
    return list(map(lambda x: str(x),names))

In [46]:
names_v = list(map(lambda x: preprocess(x),X_train))
#names_v = list(X_train)

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
vec = CountVectorizer(lowercase=True,token_pattern='.')

In [49]:
name_vec = vec.fit_transform(names_v)

In [50]:
name_vec

<9988x27 sparse matrix of type '<class 'numpy.int64'>'
	with 77906 stored elements in Compressed Sparse Row format>

In [51]:
print(vec.vocabulary_)

{'m': 13, 'a': 1, 'd': 4, 'h': 8, 'o': 15, 'r': 18, 'b': 2, 's': 19, 'u': 21, 'j': 10, 'e': 5, 't': 20, ' ': 0, 'i': 9, 'n': 14, 'g': 7, 'c': 3, 'v': 22, 'l': 12, 'k': 11, 'f': 6, 'y': 25, 'p': 16, 'w': 23, 'z': 26, 'q': 17, 'x': 24}


### Convert name

In [52]:
def name_prep(word):
    #te = re.sub(r'/.*|@.*','',word).strip()
    te = str(word).lower()
    te = re.sub(r'[^a-z.\s].*','',te).strip()
    te = re.sub(r'[^a-z ]+','',te).strip()
    te = re.sub(r' +',' ',te).strip()
    te = re.sub(r'along.*|with.*|and.*',' ',te).strip()
    #te = re.sub(r'\b\w$|\b\w\w$','',te).strip()
    te = re.sub(r'\b\w\b|\b\w\w\b','',te).strip()
    #te = re.sub(r'\bna\b|\bNaN\b','',te).strip()
    if(len(te) < 3):
        return None
    else:
        return te

In [53]:
def conv_input(name,vectorizer):
    pre = name_prep(name)
    return vectorizer.transform([pre])

## Model

In [67]:
train_score = {}
test_score = {}

In [66]:
model_1 = 'NavieB'

In [54]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [55]:
from sklearn.naive_bayes import GaussianNB

In [56]:
naive_b = GaussianNB()

In [58]:
naive_b.fit(name_vec.toarray(),y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [59]:
p = naive_b.predict(conv_input('Narendra Modi',vec).toarray())

In [60]:
map_info['gender'][p[0]]

'Male'

In [69]:
test_score[model_1] = naive_b.score(vec.transform(name_to_list(X_test)).toarray(),y_test)
test_score

{'NavieB': 0.6199439327192631}

In [70]:
train_score[model_1] = naive_b.score(name_vec.toarray(),y_train)
train_score

{'NavieB': 0.6259511413696436}